In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# AGS 069 – LLaVA Multimodal Reasoning on nuScenes Images

!pip install llava -q  # Ensure LLaVA repo is cloned if needed
!pip install nuscenes-devkit -q

from nuscenes.nuscenes import NuScenes
from PIL import Image
import requests
from io import BytesIO
from llava.model.builder import load_pretrained_model
from llava.conversation import conv_templates
import torch

# 🔍 Load nuScenes sample data
nusc = NuScenes(version='v1.0-mini', dataroot='data/nuscenes', verbose=True)
sample = nusc.sample[0]
camera_data = nusc.get('sample_data', sample['data']['CAM_FRONT'])

image_path = 'data/nuscenes/' + camera_data['filename']
image = Image.open(image_path).convert('RGB')

# 🧠 Load LLaVA model
model_name = "liuhaotian/llava-v1.5-13b"
model, vis_processors, tokenizer = load_pretrained_model(
    model_path=model_name, 
    model_base=None, 
    model_name="llava"
)

# 💬 Visual Reasoning Prompt
prompt = "Describe the traffic scene. Is there any hazard ahead? What is the weather condition?"

# 🧠 Apply image processing
image_tensor = vis_processors["eval"](image).unsqueeze(0).cuda()

# 🗣️ Multimodal Dialogue using LLaVA
conv = conv_templates["llava_v1"].copy()
conv.append_message(conv.roles[0], prompt)
conv.append_message(conv.roles[1], None)

with torch.no_grad():
    output = model.generate(
        input_ids=tokenizer.encode(conv.get_prompt(), return_tensors="pt").cuda(),
        images=image_tensor,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.2
    )

response = tokenizer.decode(output[0], skip_special_tokens=True)
print("LLaVA Scene Reasoning:\n", response)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 45.1 MB/s eta 0:00:00
  

ModuleNotFoundError: No module named 'nuscenes'